# L01: 競馬予想モデル学習（データリーク防止版）

## 修正ポイント
- 時系列を厳密に守る: 各レース時点での「過去のみ」のデータで特徴量計算
- 前走情報をメインに使用（shift で取得）

**実行環境**: ローカル（M4 MacBook Air）

---
## 1. 環境設定

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

RESULTS_PARQUET = PROJECT_ROOT / 'data' / 'processed' / 'results.parquet'
MODEL_DIR = PROJECT_ROOT / 'models'
MODEL_DIR.mkdir(exist_ok=True)

print(f"データパス: {RESULTS_PARQUET}")

In [ ]:
results_df = pd.read_parquet(RESULTS_PARQUET)
print(f"出走馬データ数: {len(results_df):,}")

---
## 2. 前処理・前走情報追加

In [ ]:
results_df['race_date'] = pd.to_datetime(results_df['race_date'])
results_df['year'] = results_df['race_date'].dt.year

# 完走馬のみ
results_df = results_df[results_df['is_finished'] == True].copy()

# NaN IDを埋める（groupbyでスキップされないように）
results_df['jockey_id'] = results_df['jockey_id'].fillna('unknown')
results_df['trainer_id'] = results_df['trainer_id'].fillna('unknown')

# 馬ごとに時系列ソート
results_df = results_df.sort_values(['horse_id', 'race_date']).reset_index(drop=True)

print(f"データ数: {len(results_df):,}")

In [ ]:
# 前走情報（shift で安全に取得）
print("前走情報を計算中...")
results_df['prev_finish'] = results_df.groupby('horse_id')['finish_position'].shift(1)
results_df['prev_last_3f'] = results_df.groupby('horse_id')['last_3f'].shift(1)
results_df['prev_odds'] = results_df.groupby('horse_id')['odds'].shift(1)
results_df['prev_race_date'] = results_df.groupby('horse_id')['race_date'].shift(1)
results_df['days_since_last'] = (results_df['race_date'] - results_df['prev_race_date']).dt.days

# 2走前、3走前
results_df['prev2_finish'] = results_df.groupby('horse_id')['finish_position'].shift(2)
results_df['prev3_finish'] = results_df.groupby('horse_id')['finish_position'].shift(3)
results_df['prev2_last_3f'] = results_df.groupby('horse_id')['last_3f'].shift(2)
results_df['prev3_last_3f'] = results_df.groupby('horse_id')['last_3f'].shift(3)

# 直近3走の平均
results_df['avg_finish_last3'] = results_df[['prev_finish', 'prev2_finish', 'prev3_finish']].mean(axis=1)
results_df['avg_last3f_last3'] = results_df[['prev_last_3f', 'prev2_last_3f', 'prev3_last_3f']].mean(axis=1)

# 新馬フラグ
results_df['is_debut'] = results_df['prev_finish'].isna().astype(int)

print("完了")

In [ ]:
# 馬の累積統計（transformで同じ長さを保証）
print("馬の累積統計を計算中...")

results_df['horse_cumulative_races'] = results_df.groupby('horse_id').cumcount()

# 勝ちフラグ（shift で過去のみ）
results_df['_win'] = (results_df.groupby('horse_id')['finish_position'].shift(1) == 1).astype(int)
results_df['_place'] = (results_df.groupby('horse_id')['finish_position'].shift(1) <= 3).astype(int)
results_df['horse_cumulative_wins'] = results_df.groupby('horse_id')['_win'].cumsum()
results_df['horse_cumulative_place'] = results_df.groupby('horse_id')['_place'].cumsum()

# 勝率・複勝率
results_df['horse_win_rate'] = results_df['horse_cumulative_wins'] / results_df['horse_cumulative_races'].replace(0, np.nan)
results_df['horse_place_rate'] = results_df['horse_cumulative_place'] / results_df['horse_cumulative_races'].replace(0, np.nan)

results_df.drop(columns=['_win', '_place'], inplace=True)
print("完了")

In [ ]:
# 騎手・調教師の累積成績
print("騎手・調教師の累積成績を計算中...")

# 日付順にソート（重要）
results_df = results_df.sort_values('race_date').reset_index(drop=True)

# 騎手
results_df['jockey_cumulative_races'] = results_df.groupby('jockey_id').cumcount()
results_df['_jwin'] = (results_df.groupby('jockey_id')['finish_position'].shift(1) == 1).astype(int)
results_df['_jplace'] = (results_df.groupby('jockey_id')['finish_position'].shift(1) <= 3).astype(int)
results_df['jockey_cumulative_wins'] = results_df.groupby('jockey_id')['_jwin'].cumsum()
results_df['jockey_cumulative_place'] = results_df.groupby('jockey_id')['_jplace'].cumsum()
results_df['jockey_win_rate'] = results_df['jockey_cumulative_wins'] / results_df['jockey_cumulative_races'].replace(0, np.nan)
results_df['jockey_place_rate'] = results_df['jockey_cumulative_place'] / results_df['jockey_cumulative_races'].replace(0, np.nan)
results_df.drop(columns=['_jwin', '_jplace'], inplace=True)

# 調教師
results_df['trainer_cumulative_races'] = results_df.groupby('trainer_id').cumcount()
results_df['_twin'] = (results_df.groupby('trainer_id')['finish_position'].shift(1) == 1).astype(int)
results_df['_tplace'] = (results_df.groupby('trainer_id')['finish_position'].shift(1) <= 3).astype(int)
results_df['trainer_cumulative_wins'] = results_df.groupby('trainer_id')['_twin'].cumsum()
results_df['trainer_cumulative_place'] = results_df.groupby('trainer_id')['_tplace'].cumsum()
results_df['trainer_win_rate'] = results_df['trainer_cumulative_wins'] / results_df['trainer_cumulative_races'].replace(0, np.nan)
results_df['trainer_place_rate'] = results_df['trainer_cumulative_place'] / results_df['trainer_cumulative_races'].replace(0, np.nan)
results_df.drop(columns=['_twin', '_tplace'], inplace=True)

print("完了")

---
## 3. データ分割

In [ ]:
TRAIN_END_YEAR = 2024

train_df = results_df[results_df['year'] <= TRAIN_END_YEAR].copy()
test_df = results_df[results_df['year'] > TRAIN_END_YEAR].copy()

train_df['surface_encoded'] = train_df['surface'].map({'芝': 0, 'ダート': 1}).fillna(-1)
test_df['surface_encoded'] = test_df['surface'].map({'芝': 0, 'ダート': 1}).fillna(-1)
train_df['target'] = (train_df['finish_position'] <= 3).astype(int)
test_df['target'] = (test_df['finish_position'] <= 3).astype(int)

print(f"Train: {len(train_df):,}")
print(f"Test: {len(test_df):,}")

---
## 4. モデル学習

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

FEATURE_COLS_NO_ODDS = [
    'distance', 'surface_encoded', 'level_score',
    'horse_number', 'gate_number', 'impost',
    'prev_finish', 'prev_last_3f', 'days_since_last', 'is_debut',
    'prev2_finish', 'prev3_finish',
    'avg_finish_last3', 'avg_last3f_last3',
    'horse_cumulative_races', 'horse_win_rate', 'horse_place_rate',
    'jockey_cumulative_races', 'jockey_win_rate', 'jockey_place_rate',
    'trainer_cumulative_races', 'trainer_win_rate', 'trainer_place_rate',
]

FEATURE_COLS_WITH_ODDS = FEATURE_COLS_NO_ODDS + ['popularity', 'odds']

print(f"オッズなし: {len(FEATURE_COLS_NO_ODDS)}個")
print(f"オッズあり: {len(FEATURE_COLS_WITH_ODDS)}個")

In [ ]:
for col in FEATURE_COLS_WITH_ODDS:
    if col in train_df.columns:
        train_df[col] = train_df[col].fillna(0)
        test_df[col] = test_df[col].fillna(0)

train_clean = train_df.dropna(subset=['target'])
test_clean = test_df.dropna(subset=['target'])

print(f"Train: {len(train_clean):,}, Test: {len(test_clean):,}")

In [ ]:
def train_and_evaluate(train_data, test_data, feature_cols, model_name):
    print(f"\n{'='*50}")
    print(f"🎯 {model_name}")
    print(f"{'='*50}")
    
    X_train = train_data[feature_cols]
    y_train = train_data['target']
    X_test = test_data[feature_cols]
    y_test = test_data['target']
    
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 15,
        'min_data_in_leaf': 100,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'verbose': -1,
        'random_state': 42,
    }
    
    model = lgb.train(
        params,
        lgb.Dataset(X_tr, label=y_tr),
        num_boost_round=1000,
        valid_sets=[lgb.Dataset(X_tr, label=y_tr), lgb.Dataset(X_val, label=y_val)],
        valid_names=['train', 'valid'],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )
    
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    print(f"\n📊 Test AUC: {auc:.4f}")
    
    test_with_pred = test_data.copy()
    test_with_pred['pred_proba'] = y_pred
    
    # 的中率
    race_hit = test_with_pred.groupby('race_id').apply(
        lambda g: (g.nlargest(3, 'pred_proba')['finish_position'] <= 3).any()
    )
    print(f"🎯 上位3頭的中率: {race_hit.mean():.1%}")
    
    # 単勝回収率
    returns = test_with_pred.groupby('race_id').apply(
        lambda g: g.nlargest(1, 'pred_proba').iloc[0]['odds'] * 100 
                  if g.nlargest(1, 'pred_proba').iloc[0]['finish_position'] == 1 else 0
    )
    return_rate = returns.sum() / (len(returns) * 100) * 100
    print(f"💰 単勝回収率: {return_rate:.1f}%")
    
    # 三連複
    sanren = test_with_pred.groupby('race_id').apply(
        lambda g: len(set(g.nsmallest(3, 'finish_position')['horse_id']) & 
                      set(g.nlargest(5, 'pred_proba')['horse_id'])) >= 3
    )
    print(f"🏇 三連複的中率（5頭選択）: {sanren.mean():.1%}")
    
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importance()
    }).sort_values('importance', ascending=False)
    
    return {'model': model, 'auc': auc, 'hit_rate': race_hit.mean(), 
            'return_rate': return_rate, 'sanren_rate': sanren.mean(), 
            'importance': importance, 'predictions': test_with_pred}

In [ ]:
result_no_odds = train_and_evaluate(train_clean, test_clean, FEATURE_COLS_NO_ODDS, "オッズなしモデル")

In [ ]:
result_with_odds = train_and_evaluate(train_clean, test_clean, FEATURE_COLS_WITH_ODDS, "オッズありモデル")

---
## 5. 比較・保存

In [ ]:
print("\n" + "="*60)
print("📊 モデル比較")
print("="*60)

print(pd.DataFrame({
    '指標': ['AUC', '上位3頭的中率', '三連複的中率', '単勝回収率'],
    'オッズなし': [f"{result_no_odds['auc']:.4f}", f"{result_no_odds['hit_rate']:.1%}", 
                   f"{result_no_odds['sanren_rate']:.1%}", f"{result_no_odds['return_rate']:.1f}%"],
    'オッズあり': [f"{result_with_odds['auc']:.4f}", f"{result_with_odds['hit_rate']:.1%}",
                   f"{result_with_odds['sanren_rate']:.1%}", f"{result_with_odds['return_rate']:.1f}%"]
}).to_string(index=False))

In [ ]:
print("=== 特徴量重要度 (オッズなし) ===")
print(result_no_odds['importance'].to_string(index=False))

In [ ]:
import pickle

with open(MODEL_DIR / 'model_no_odds.pkl', 'wb') as f:
    pickle.dump(result_no_odds['model'], f)
with open(MODEL_DIR / 'model_with_odds.pkl', 'wb') as f:
    pickle.dump(result_with_odds['model'], f)

config = {
    'feature_cols_no_odds': FEATURE_COLS_NO_ODDS,
    'feature_cols_with_odds': FEATURE_COLS_WITH_ODDS,
    'train_end_year': TRAIN_END_YEAR,
}
with open(MODEL_DIR / 'model_config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("✅ モデル保存完了")